In [1]:
import numpy as np
from numpy import genfromtxt
from itertools import combinations
import torch
import math 
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
def angle_btw_hits(arr1,arr2):
    theta1 = arr1[2]
    theta2 = arr2[2]
    phi1 = arr1[3]
    phi2 = arr2[3]
    opang = math.acos(math.sin(theta1)*math.sin(theta2)*math.cos(phi1-phi2)+math.cos(theta1)*math.cos(theta2))
    return opang

In [3]:
def flatten(xss):
    return [x for xs in xss for x in xs]

In [4]:
def run_r3b_clustering(data,clustersize):
    print(data.shape)
    array_unique_events = np.unique(data[:,0])
    print(array_unique_events)
    all_pred = []
    for i in range(0,int(np.max(data,axis=0)[0]+1)-(int(np.max(data,axis=0)[0]+1)%3) ,3):
    #for i in range(0,int(np.max(data,axis=0)[0])-(int(np.max(data,axis=0)[0])%3) ,3):
        evt = data[data[:,0] == array_unique_events[i]]
        evt = np.vstack((evt,data[data[:,0] == array_unique_events[i+1]]))
        evt = np.vstack((evt,data[data[:,0] == array_unique_events[i+2]]))
        nr_hits = evt.shape[0]
        hitnr = np.array([x for x in range(evt.shape[0])])
        hitnr = np.transpose(hitnr)
        hitnr = hitnr.reshape(-1, 1)
        evt = np.hstack((evt,hitnr))
        # sort along the energy
        evt = evt[evt[:, 1].argsort()[::-1]]
        #now check opening angle between the hits
        cluster_list = []
        while(evt.shape[0]):
            v_temp = []
            l = []
            for i in range(evt.shape[0]):
                if (i == 0):
                    l.append(evt[0,5].astype(int))
                else:
                    #calculate angle
                    angle = angle_btw_hits(evt[0,:],evt[i,:])
                    if (angle < clustersize):
                        l.append(evt[i,5].astype(int))
                    else:
                        v_temp.append(evt[i,:])
            l.sort()
            cluster_list.append(l)
            evt = np.array(v_temp)
        #compress list of clusters, they need at least two entries to be a cluster, otherwise the single hit is treated as cluster
        compr_cluster_list = []
        for i in range(len(cluster_list)):
            if len(cluster_list[i]) > 1:
                compr_cluster_list.append(cluster_list[i])
            else:
                continue
        final_pair_list = []
        for i in range(len(compr_cluster_list)):
            pair_indices = list(combinations(compr_cluster_list[i],2))
            final_pair_list.append(pair_indices)
        reco_indices = flatten(final_pair_list)
        # compare all combinations with all predicted combinations
        all_combinations = list(combinations(range(nr_hits), 2))
        ## predictions
        pred_list = []
        for i in range(len(all_combinations)):
            pred_value = 0
            for j in range(len(reco_indices)):
                if (all_combinations[i] == reco_indices[j]):
                    #print(type(reco_indices[j]))
                    pred_value = 1
            pred_list.append(pred_value)
            all_pred.append(pred_value)
    return all_pred
        




        

## now method to plot data:

In [5]:
def plot_true_reco(data):
    energy_spec = []
    unique_ind = np.unique(data[:,0])
    for i in unique_ind:
        ene = np.sum(data[data[:,0] == i,1])
        energy_spec.append(ene)

    plt.hist(energy_spec,bins=70,range=(0,7))
    plt.title("True energy spectrum")
    plt.yscale('log')


### in this section I want to get the true values combination list
---

In [6]:
def get_true_combinatorics(data):
    ll = []
    true_vals = []
    for i in range(0,int(np.max(data,axis=0)[0]+1)-(int(np.max(data,axis=0)[0]+1)%3) ,3):
    #for i in range(0,int(np.max(data,axis=0)[0])-(int(np.max(data,axis=0)[0])%3) ,3):
        mask = ((data[:,0] == i) | (data[:,0] == i+1) | (data[:,0] == i+2))
        evt = data[mask]
        pair_indices = list(combinations(range(evt.shape[0]), 2))
        for i, (idx1, idx2) in enumerate(pair_indices):
            l = []
            if (evt[idx1,0] == evt[idx2,0]):
                true_vals.append(1)
            else:
                true_vals.append(0)
    return (true_vals)

---

In [7]:
def get_true_reco_ene(data,reco_vals,true_vals):
    #print("reco_vals",reco_vals)
    ene_list = []
    ene_true_list = []
    idx = 0
    well_reco = 0
    array_unique_events = np.unique(data[:,0])
    nr_events = 0
    nr_t = 0
    nr_fp = 0
    nr_fn = 0
    nr_fpfn = 0
    for i in range(0,int(np.max(data,axis=0)[0]+1)-(int(np.max(data,axis=0)[0]+1)%3) ,3):
    #for i in range(0,int(np.max(data,axis=0)[0])-(int(np.max(data,axis=0)[0])%3) ,3):
        nr_events += 1
        mask = ((data[:,0] == i) | (data[:,0] == i+1) | (data[:,0] == i+2))
        evt = data[mask]
        pair_indices = list(combinations(range(evt.shape[0]), 2))
        ##pair indices which belong together...
        pred_comb = []
        ##TJ this is the new part with true stuff
        true_comb = []
        for m in range(evt.shape[0]):
            pred_comb.append((m,m))
            true_comb.append((m,m))
        for j in range (len(pair_indices)):
            #print("reco_vals[idx+j]:\t", reco_vals[idx+j])
            if reco_vals[idx+j] > 0.75:
                pred_comb.append(pair_indices[j])
            if true_vals[idx+j] > 0.75:   
                true_comb.append(pair_indices[j])
        G = nx.Graph()
        G.add_edges_from(pred_comb)
        connected_components = list(nx.connected_components(G))
        ##TJ again, this it the addition for the true values
        T = nx.Graph()
        T.add_edges_from(true_comb)
        true_components = list(nx.connected_components(T))
        set_true = {frozenset(sublist) for sublist in true_components}
        set_pred = {frozenset(sublist) for sublist in connected_components}

        evt_type =  check_matrix(set_true,set_pred)
        if (evt_type == "t"):
            nr_t += 1
        if (evt_type == "fp"):
            nr_fp += 1
        if (evt_type == "fn"):
            nr_fn +=1
        if (evt_type == "fpfn"):
            nr_fpfn += 1        
        
        #print("connected components:\t", connected_components)        
        # Find the intersection of the two sets
        common_sublists = set_true.intersection(set_pred)

        well_reco += len(common_sublists)
        for k in range(len(connected_components)):
            columns_to_sum = evt[list(connected_components[k]),1]
            ene = np.sum(columns_to_sum)
            ene_list.append(ene)
        #make list true energy
        for l in range(len(true_components)):
            true_columns_to_sum = evt[list(true_components[l]),1]
            ene_true = np.sum(true_columns_to_sum)
            ene_true_list.append(ene_true)
        idx += len(pair_indices)
    single_hit_energies = []
    for i in range(data.shape[0]):
        single_hit_energies.append(data[i,1])
        
    #plt.hist(single_hit_energies,bins=70,range=(0,7),label="single hit energy",color="green",alpha=0.5)    
    plt.hist(ene_list,bins=100,range=(0,10),label="reconstructed energy",color="red",alpha=0.3)
    plt.grid()
    plt.hist(ene_true_list,bins=100,range=(0,10),label="true energies",color="blue",alpha=0.3)
    plt.legend()
    plt.yscale('log')
    well_reco = float(well_reco)/float(np.max(data,axis=0)[0])

    print("well reco...", well_reco)
    print("---more statistics....---")
    print("fully true_reco_cluster:\t", nr_t/nr_events)
    print("false positive events:\t" , nr_fp/nr_events)
    print("false negative events:\t", nr_fn/nr_events)
    print("mixed false events:\t" , nr_fpfn/nr_events)
    print("end of statistiscs------")
    return (ene_list,ene_true_list,well_reco)

In [8]:
# #data = genfromtxt('raw_data_test.txt', delimiter=',')
# #data = genfromtxt('validation_raw.txt',delimiter=',')
# data = genfromtxt('data_raw_uniform_test.csv',delimiter=',')
# #data = genfromtxt('data_stream_2121.txt', delimiter=',')
# #my_data[:,4] = my_data[:,4]+4500  #this step is needed, I only want positive time values, so that I can use the time as a radius
# data[:,4] = 1  #this step is needed, I only want positive time values, so that I can use the time as a radius
# # ### structure of mydata : eventnr, energy, theta, phi, hit-time
# data = data*[1.,1.,3.14159/180,3.14159/180,1.]
# unique_values, continuous_numbers = np.unique(data[:,0], return_inverse=True)
# data[:,0] = continuous_numbers
# print("continuous numbers:", continuous_numbers)
# out_cluster_r3b = run_r3b_clustering(data,0.25)
# #foo = plot_true_reco(data)
# foo_comb = get_true_combinatorics(data)
# foo_ene = get_true_reco_ene(data,out_cluster_r3b,foo_comb)



### functionality to write clustered data to file
#### e.g hit1, hit2, hit3, hit4,
#### if hit1, hit2 belong together then rewrite hit1 and hit 2 that they have both summed energy and same theta, phi (from cms)

In [9]:
def sph2cart(data):
    r = data[:,1]
    rsin_theta = r*np.sin(data[:,2])
    x = rsin_theta*np.cos(data[:,3])
    y = rsin_theta*np.sin(data[:,3])
    z = r*np.cos(data[:,2])
    cart_data = np.array([data[:,0],data[:,1],x,y,z,data[:,4]]).T
    #return np.array([data[:,0],data[:,1],x,y,z,data[:,4]])
    return cart_data

In [10]:
def func_cm(data):
    #print("this should be my cartesian data...")
    #print(data)
    #print("and the shape of the data is:\t", data.shape)
    positions = data[:,2:5]
    #print("positions")
    #print(positions)
    masses = data[:,1]
    #print("masses")
    #print(masses)
    #print(data.shape[0])
    sum_mass = (np.repeat(np.sum(masses),data.shape[0])).reshape(-1,1)
    #print("shape of sum masses:\t", sum_mass.shape)
    cm = np.sum(positions.T * masses, axis=1) / np.sum(masses)
    #cm = cm.reshape(-1,3)
    cm_x = (np.repeat(cm[0],data.shape[0])).reshape(-1,1)
    cm_y = (np.repeat(cm[1],data.shape[0])).reshape(-1,1)
    cm_z = (np.repeat(cm[2],data.shape[0])).reshape(-1,1)
    #print("cm",cm)
    time = data[np.argmax(data[:,1],axis=0),5]
    time = (np.repeat(time,data.shape[0])).reshape(-1,1)
    index_evt = (data[:,0]).reshape(-1,1)
    #print("shape of data[:,0]:\t", index_evt.shape)
    #print("shape of sum_mass:\t", sum_mass.shape)
    #print("shape of cm_x:\t", cm_x.shape)
    #print("shape of cm_y:\t", cm_y.shape)
    #print("shape of cm_z:\t", cm_z.shape)
    #print("shape of time:\t", time.shape)
    arr_cart = np.concatenate((index_evt,sum_mass,cm_x,cm_y,cm_z,time), axis=1)
    #print("incoming data struc:")
    #print(data)
    #print("outgoing cms data:")
    #print(arr_cart)
    return arr_cart


In [11]:
def cart2sph(data):
    x = data[:,2]
    y = data[:,3]
    z = data[:,4]
    hxy = np.hypot(x, y)
    r = np.hypot(hxy, z)
    th = ((np.arccos(z/r))/math.pi)*180
    az = ((np.arctan2(y,x))/math.pi)*180 
    az[az < 0] += 360
    return_arr = np.array([data[:,0],data[:,1],th,az,data[:,5]]).T
    return return_arr
    
    

In [12]:
def write_reco_to_file_cms(data,reco_vals):
    #print("reco_vals",reco_vals)
    ene_list = []
    ene_true_list = []
    idx = 0
    array_unique_events = np.unique(data[:,0])
    fmt = '%i','%1.5f', '%1.5f','%1.5f','%1.2f' 
    with open('test_file_agllo_13_02.txt', 'w') as f:
        for i in range(0,int(np.max(data,axis=0)[0]+1)-(int(np.max(data,axis=0)[0]+1)%3) ,3):
        #for i in range(0,int(np.max(data,axis=0)[0])-(int(np.max(data,axis=0)[0])%3) ,3):
            mask = ((data[:,0] == i) | (data[:,0] == i+1) | (data[:,0] == i+2))
            evt = data[mask]
            evt[:,4] = evt[:,4] - 4500
            pair_indices = list(combinations(range(evt.shape[0]), 2))
            ##pair indices which belong together...
            pred_comb = []
            ##TJ this is the new part with true stuff
            true_comb = []
            for m in range(evt.shape[0]):
                pred_comb.append((m,m))
                true_comb.append((m,m))
            for j in range (len(pair_indices)):
                #print("reco_vals[idx+j]:\t", reco_vals[idx+j])
                if reco_vals[idx+j] > 0.75:
                    pred_comb.append(pair_indices[j])
            G = nx.Graph()
            G.add_edges_from(pred_comb)
            connected_components = list(nx.connected_components(G))
            set_pred = {frozenset(sublist) for sublist in connected_components}
            #print("set_pred", set_pred)
            #print("connected components:\t", connected_components)
            for k in range(len(connected_components)):
                ## do cms for all clustered hits and write everything to file
                cluster_data = evt[list(connected_components[k]),:]
                #print(cluster_data)
                cluster_data_cart = sph2cart(cluster_data)
                #print("cartesian data")
                #print(cluster_data_cart)
                cluster_data_cart_cms = func_cm(cluster_data_cart)
                cluster_data_sph_cms = cart2sph(cluster_data_cart_cms)
                #print("this is the shape of the data I want to store in file:",cluster_data_sph_cms.shape)
                #now write to file..
                #print("data shape...",data.shape)
                for l in range(cluster_data_sph_cms.shape[0]):
                    #print("llll:\t",l)
                    np.savetxt(f,cluster_data_sph_cms[l,:].reshape(1,-1),fmt = fmt,delimiter=",")
            idx += len(pair_indices)
    f.close()

            

In [13]:
import copy
def check_matrix(f_set1, f_set2):
    #print("input true:\t", f_set1)
    #print("input pred:\t", f_set2)
    set1 = copy.deepcopy(f_set1)
    set2 = copy.deepcopy(f_set2)
    #logic here inside
    # Remove identical frozensets
    common = set1 & set2
    set1 -= common
    set2 -= common

    # Step 2: Remove subsets from set1 if they are subsets of elements in set2
    set1 = {s1 for s1 in set1 if not any(s1.issubset(s2) for s2 in set2)}

    # Step 3: Remove subsets from set2 if they are subsets of elements in set1
    set2 = {s2 for s2 in set2 if not any(s2.issubset(s1) for s1 in set1)}

    #print("Length of set1:\t", len(set1))
    #print("Length of set2:\t", len(set2))

    # Final result
    #print("Updated set1:", set1)
    #print("Updated set2:", set2)

    if (len(set1) == 0 and len(set2) == 0):
        return  "t"
    if (len(set1) == 0 and len(set2) != 0):
        return "fp"
    if (len(set1) != 0 and len(set2) == 0):
        return "fn"
    if (len(set1) != 0 and len(set2) != 0):
        return "fpfn"